# Financial AI Agent
***
**An AI-powered conversational assistant that helps retail banking customers understand their financial behavior through natural language queries.**
***

This project implements a production-grade Financial AI Agent that:
* Understands natural language financial questions
* Executes queries against transaction data with accurate calculations
* Explains answers in simple, conversational language
* Logs complete reasoning trails for compliance and observability
***
⚠️ Note on Data: 
* transactions.csv contains partial, synthetically generated data for demonstration purposes only. It is not real banking data. In production, this would connect to actual transaction databases.

In [1]:
# import project_structure as ps
# ps.show_project_structure()

In [2]:
# ============================================================================
# STANDARD LIBRARY IMPORTS
# ============================================================================

import os
import sys
import json
from pathlib import Path
from datetime import date, datetime, timedelta
from typing import Dict, Any, List, Optional
from pprint import pprint

# ============================================================================
# THIRD-PARTY IMPORTS
# ============================================================================

import pandas as pd
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph

# ============================================================================
# PROMPTS (LLM-1 & LLM-2)
# ============================================================================

from prompts.llm1_prompt import OPTIMIZED_ROUTER_SYSTEM_PROMPT
from prompts.llm2_prompt import llm2_prompt_builder, BASE_LLM2_SYSTEM_PROMPT

# print("✅ Prompts imported successfully")

# ============================================================================
# SCHEMAS & MODELS
# ============================================================================

from schemas.router_models import (
    GraphState,
    RouterOutput,
    ConversationSummary,
    PreferenceEntry,
    ExecutionResult,
    BackofficeLog,
    DataSources,
    ClarificationStep
)

from schemas.executor_models_llm2 import (
    TransactionQuerySpec,
    TransactionQueryResult,
    TransactionRecord,
)

# print("✅ Schemas & models imported successfully")

# ============================================================================
# GRAPH DEFINITION (LangGraph Nodes & Edges)
# ============================================================================

from graph_definition import (
    build_graph,
    router_node,
    vague_handler_node,
    executor_node,
    summary_update_node,
    build_router_payload,
    build_executor_payload
)

# print("✅ Graph definition imported successfully")

# ============================================================================
# TOOLS
# ============================================================================

from schemas.transactions_tool import (
    query_transactions_lc_tool, 
    query_transactions_tool
)

# print("✅ Tools imported successfully")

# ============================================================================
# BACK-OFFICE LOGGING
# ============================================================================

import backoffice_logging as bolog

# print("✅ Back-office logging imported successfully")

# ============================================================================
# TESTS
# ============================================================================

# import tests.llm1_tests as tst_llm1
# import tests.llm2_tests as tst_llm2
# import tests.pipeline_no_rag_tests as tst_no_rag_pipeline

# print("✅ Test modules imported successfully")

# Configurations 

In [3]:
# Enable autoreload for development
%load_ext autoreload
%autoreload 2

In [4]:
"""
Create prompts/__init__.py
Makes prompts/ directory a proper Python package
"""

from pathlib import Path

# Define the complete content
INIT_CONTENT = """\"\"\"
Prompt modules for LLM-1 (Router & Clarifier) and LLM-2 (Executor)
\"\"\"

from .llm1_prompt import OPTIMIZED_ROUTER_SYSTEM_PROMPT
from .llm2_prompt import llm2_prompt_builder, BASE_LLM2_SYSTEM_PROMPT

__all__ = [
    'OPTIMIZED_ROUTER_SYSTEM_PROMPT',
    'llm2_prompt_builder',
    'BASE_LLM2_SYSTEM_PROMPT',
]
"""

# Create the file
project_root = Path.cwd()
init_file = project_root / "prompts" / "__init__.py"

print("=" * 80)
print("Creating prompts/__init__.py")
print("=" * 80)

# Write file
with open(init_file, 'w', encoding='utf-8') as f:
    f.write(INIT_CONTENT)

# # Verify
# if init_file.exists():
#     file_size = init_file.stat().st_size
#     print(f"\n✅ SUCCESS: File created")
#     print(f"   Location: {init_file}")
#     print(f"   Size: {file_size} bytes")
    
#     # Show contents
#     print("\n📄 File Contents:")
#     print("-" * 80)
#     with open(init_file, 'r', encoding='utf-8') as f:
#         print(f.read())
#     print("-" * 80)
    
#     print("\n" + "=" * 80)
#     print("✅ DONE! Now restart kernel and re-run imports")
#     print("=" * 80)
# else:
#     print("\n❌ ERROR: File was not created")


Creating prompts/__init__.py


In [5]:
# Load autoreload extension
%load_ext autoreload

# Set autoreload mode
# Mode 2: Reload all modules (except those excluded) every time before executing code
%autoreload 2
    
print("✅ Autoreload enabled (mode 2)")
print("   All modules will be automatically reloaded when changed")
print("\n📝 Watched modules:")
print("   • All modules in schemas/")
print("   • All modules in prompts/")
print("   • All modules in tests/")
print("   • Changes will be reflected immediately without kernel restart")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Autoreload enabled (mode 2)
   All modules will be automatically reloaded when changed

📝 Watched modules:
   • All modules in schemas/
   • All modules in prompts/
   • All modules in tests/
   • Changes will be reflected immediately without kernel restart


# ***************************************************************************************

# TEST-1: Schema Validation for all Pydantic models 
***
* Validate all Pydantic models load correctly before running LLM or RAG tests.
* No LLM calls, no API keys needed - runs in <1 second.
***
**Schemas Tested:**
* **PreferenceEntry** - User preferences with override tracking (previous_value, previous_turn_id)
* **ConversationSummary** - Session-scoped preferences (time_window, amount_threshold_large)
* **RouterOutput** - LLM-1 structured output (clarity, core_use_cases, summary_update)
* **BackofficeLog** - Compliance logging with reasoning_steps and data_sources
* **GraphState** - LangGraph state with raw_messages for multi-turn
***

In [6]:
import tests.test_schemas as ts
ts.test_all_schemas()

🧪 SCHEMA VALIDATION TESTS

✅ PreferenceEntry
   Fields: value, source, turn_id, original_query, previous_value, previous_turn_id

✅ ResolvedDates
   Fields: start_date, end_date, interpretation

✅ ConversationSummary
   Fields: time_window, amount_threshold_large, account_scope, category_preferences

✅ RouterOutput
   Fields: clarity, core_use_cases, uc_operations, primary_use_case, ...
   Fields: complexity_axes, needed_tools, clarifying_question, missing_info, ...
   Fields: summary_update, resolved_dates, resolved_trn_categories, resolved_amount_threshold

✅ DataSources
   Fields: tables_used, fields_accessed, filters_applied, aggregations_used

✅ ClarificationStep
   Fields: question, user_answer, turn_id

✅ BackofficeLog
   Fields: user_query, resolved_query, answer, analysis, reasoning_steps, ...
   Fields: data_sources, transactions_analyzed, preferences_used, ...
   Fields: clarification_history, confidence, rag_used, router_output_snapshot

✅ ExecutionResult
   Fields: final_a

[('PreferenceEntry', True, None),
 ('ResolvedDates', True, None),
 ('ConversationSummary', True, None),
 ('RouterOutput', True, None),
 ('DataSources', True, None),
 ('ClarificationStep', True, None),
 ('BackofficeLog', True, None),
 ('ExecutionResult', True, None),
 ('GraphState', True, None)]

# TEST-2: LLM-1 Multi-Turn (VAGUE Queries only)
***
Test LLM-1 router_node and summary_update_node behavior with multi-turn flow.
***
**What This Validates:**
* **Turn 1 VAGUE** - Clarifying question generated for missing time_window or amount_threshold
* **Turn 2 CLEAR** - Correct core_use_cases and primary_use_case after user answers
* **summary_update** - LLM-1 produces preference updates from user clarification
* **conversation_summary** - Preferences merged correctly via summary_update_node
* **raw_messages** - Full conversation history passed to LLM-1 in Turn 2
***
***
**LangGraph Coverage:**
* ✅ router_node() execution
* ✅ summary_update_node() execution  
* ✅ State passed correctly between nodes
* ❌ Full graph.compile() - tested in pipeline tests
* ❌ Conditional edges (VAGUE→vague_handler, CLEAR→executor) - tested in pipeline tests
* ❌ app.invoke() end-to-end - tested in pipeline tests
***
**Why Partial Coverage of LangGraph:** 
These tests isolate LLM-1 behavior by calling nodes directly. 
Full orchestration requires pipeline_tests.py which tests app.invoke() flow.
***

In [7]:
import tests.llm1_tests as tst_llm1

# # Test VAGUE queries 

# # 1: All queries (default)
vague_results = tst_llm1.llm1_test_all_vague_queries_multiturn()

# # 2: Random N queries
# vague_results = tst_llm1.llm1_test_all_vague_queries_multiturn(num_examples_to_check=2)
# clear_results = tst_llm1.llm1_test_all_clear_queries(num_examples_to_check=3)

# 3: Specific queries by ID
# vague_results = tst_llm1.llm1_test_all_vague_queries_multiturn(query_ids=[14, 15])

📋 Testing all 5 VAGUE queries
🧪 LLM-1 MULTI-TURN TEST: VAGUE QUERIES

Each conversation is INDEPENDENT with fresh ConversationSummary

VAGUE Query #11: "What are my coffee shop expenses?"
Missing: timeframe

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ TURN 1: User Query (VAGUE)                                                                       │
└──────────────────────────────────────────────────────────────────────────────────────────────────┘

👤 User: "What are my coffee shop expenses?"
📝 ConversationSummary: (empty - fresh conversation)

--- LLM-1 Router Iteration 1 ---


C:\Users\USER\miniconda3\envs\p3.12_ml_env\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


LLM-1 wants to use 1 tool(s)
Executing tool: search_transaction_categories with args: {'terms': ['coffee shop']}

--- LLM-1 Router Iteration 2 ---
LLM-1 responded without tool calls - parsing RouterOutput

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ TURN 1: LLM-1 Response (Clarification)                                                           │
└──────────────────────────────────────────────────────────────────────────────────────────────────┘

🤖 LLM-1 Classification:
   clarity: VAGUE
   clarifying_question: For what time period would you like to know your coffee shop expenses? For example, last month, this year, or last 30 days?
   missing_info: ['time_window']

────────────────────────────────────────────────────────────────────────────────────────────────────
  TURN 1 VALIDATION
────────────────────────────────────────────────────────────────────────────────────────────────────
   ✅ clarity: VAGUE
   ✅ clarifying_question

# ***********************************  RAG IMPLEMENTATION  *********************************

# RAG IMPLEMENTATION: Build Vector Store (for Transactions Categories)

***

### Architecture

- **Vector DB:** **ChromaDB** (persistent, local, free)  
- **Embedding model:** `multilingual-e5-base`  
- **Corpus:** **111 category documents** (20 category groups + 91 subcategories)
- **Summary:** ChromaDB + multilingual embeddings enables **fast, explainable category matching** based on semantic meaning.
  
***
## 1- Embedding model
***

Build a **semantic search vector store** for transaction category mapping (UC-04).  
We embed **conceptual descriptions** of categories (what they mean and how people use them), not exhaustive merchant/brand lists. This enables robust matching for **unseen brands** and **multilingual queries**.

***

### Key embedding decisions

- **Model:** `multilingual-e5-base` (multilingual, strong retrieval semantics, Hebrew support)
  - ✅ **Semantic understanding:** maps meaning (e.g., “Starbucks” → *Cafes & Coffee Shops*) even if not explicitly listed  
  - ✅ **Multilingual:** supports Hebrew merchant names / phrases (e.g., Aroma, Shufersal, Delek)  
  - ✅ **Retrieval-optimized:** trained for semantic search (our use case)  
  - ✅ **Practical for this KB size:** ~560MB, fast load, good fit for ~111 documents  
  - ✅ **Offline + free:** no API cost, works locally

- **Document strategy:** one document per category entry (group + subcategory) with:
  - *Name + ID + level (group/subcategory) + short description + example phrases/synonyms*
  - Full metadata preserved (20 groups + 91 subcategories)

- **Similarity threshold:** **0.75** (standard working threshold for e5-style retrieval; documented in config)

### Output artifacts

- Persistent vector store: `data/chroma_trn_categories/` (~20MB)
- 111 embedded documents with metadata + semantic descriptions

***

### Production alternative (for embedding model)

- **OpenAI `text-embedding-3-small`** (API-based)
  - **When to use:** enterprise deployments that prioritize embedding quality and can accept API cost/latency

---

## 2- Vector Store & ChromaDB (short context)

A **vector store** saves embeddings (vectors) and enables **similarity search** by meaning (not keywords).

- Traditional DB: exact filters (e.g., `WHERE category = 'Coffee'`)
- Vector search: “find the most semantically similar category to this phrase”

**How it works:**
1. Text → embedding vector  
2. Store vectors + metadata  
3. Query → retrieve nearest vectors (most similar meaning)

**Why ChromaDB for this PoC:**
- Persistent on disk (no rebuild)
- Local (no server), small/medium corpora
- Free + simple
- Fast for small KB (<1M vectors), perfect for ~111 categories

### Production alternative (for Vector Store)
- **Pinecone**
    * Large-scale cloud (Overkill for this PoC)

***

## 1-BUILD a new/ LOAD existing / or REBUILD existing Embedding model (vector store)
----

### build_vector_store()

**Model Download:** First time only (~560MB, then cached)  
**Vector Store:** Can be rebuilt anytime with force_rebuild=True

*** 

**DEFAULT => build_vector_store(force_rebuild=False):**
* ✅ Validates CategoriesKB.json exists
* ✅ Loads embedding model (cached, instant)
* ✅ Checks if vector store exists
* ✅ **Loads existing store from disk**
* ✅ Returns vectorstore object
* 
* Result: Fast: ~1-2 seconds
* **When to use: Normal operations, testing, querying. No changes in Embedding model. No changes in data (transaction categories/subcategories)**

***

**force_rebuild=True (REBUILD)**
* ✅ Validates CategoriesKB.json exists
* ✅ Loads embedding model
* ✅ **Deletes existing store**
* ✅ **Creates embedding text for 111 categories**
* ✅ **Regenerates all 111 embeddings**
* ✅ **Builds vector store**
* ✅ **Saves to disk**
* 
* Result: Slow (~30-60 seconds) - rebuilds everything
* **When to use: Updated data, changed model, fixing issues**

---
---
### `@lru_cache` - Function Result Caching 
* Remembers function results to avoid reloading expensive resources.

---
Our Use Case:
    - We cache **2 expensive resources** that must load only ONCE per session:
        - 1. Embedding Model: `intfloat/multilingual-e5-base`
        - 2. Vector Store: ChromaDB Client
        
    - Why only ONCE per session:
        - Our functions have **no arguments** (always same input)
        - Only need to cache **1 result** (the loaded model/database)
        - Minimal memory overhead
---  
Impact on Performance:
    - Single query    =>  Without Cache: 5 seconds           VS  With Cache: 5 seconds 
    - 60 queries      =>  Without Cache: 300 seconds (5 min) VS  With Cache: 6 seconds
    - Speedup Summary =>  Without Cache: 1x                  VS  With Cache **50x faster** 🚀 
---
Summary - Caching:
    - @lru_cache in our project:
    - Loads `intfloat/multilingual-e5-base` ONCE (`maxsize=1`) (560MB)
    - Loads ChromaDB collection ONCE  (`maxsize=1`) (111 categories)
    - 50x faster for multiple queries
    - Essential for production ML systems 

In [8]:
from build_category_vectorstore import build_vector_store

# # LOAD existing Embedding model (vector store):
vectorstore = build_vector_store(force_rebuild=False)  # Default

# # # Force REBUILD Embedding model (vector store):
# vectorstore = build_vector_store(force_rebuild=True) 

🏗️  BUILDING CATEGORY RAG VECTOR STORE

📂 Loading categories from: data/CategoriesKB.json
   ✅ Loaded 20 category groups
   ✅ Loaded 88 subcategories
   ✅ Total categories: 108

🧠 Loading embedding model: intfloat/multilingual-e5-base...
   ⏳ First time: downloading model (~560MB)...
   ⏳ Subsequent runs: instant loading (cached)
   ✅ Embedding model loaded

✅ Vector store already exists at: data\chroma_trn_categories
   Use force_rebuild=True to rebuild from scratch

✅ VECTOR STORE LOADED SUCCESSFULLY!
📁 Location: data\chroma_trn_categories
🔍 Collection: transaction_categories
🧠 Model: intfloat/multilingual-e5-base
💡 Ready for queries via trn_category_rag.query_categories()


# ***********************************  EMBEDDING & RAG TESTS (Optiomal) *********************************

---
## RAG - TEST 1: Vector Store Embeddings Quality (the FOUNDATION test)

* Component: build_category_vectorstore.py
* Validates: 
* Vector store directory exists
* ChromaDB loads successfully
* 111 documents present (20 groups + 91 subcategories)
* Metadata structure correct
---
This is the FOUNDATION test. If this fails, nothing else will work because:
* ❌ No vector store → Can't query categories
* ❌ Wrong count → Missing categories won't be found
* ❌ Bad metadata → Queries return incomplete data
Think of it as: Testing that the database was created successfully before running queries.
---

In [ ]:
# from tests.test_rag_1_infrastructure import test_vector_store_embeddings_quality

# # Run Test 1 only
# results = test_vector_store_embeddings_quality()

# # Check results
# print(f"\nTest 1 Result: {results['passed']}/{results['total']} checks passed")
# print(f"Accuracy: {results['accuracy']*100:.1f}%")

## RAG - TEST 2: Exact Name Retrieval

* Component: trn_category_rag.py → query_categories()
* Validates: 
    * Basic vector similarity search works
    * Exact category names retrieve correctly
    * Distance scores low (< 0.4) for exact matches
    * 8 test queries (4 groups + 4 subcategories)
---
* Master Function: run_infrastructure_tests()
* Runs both tests
* Provides combined summary
* Returns structured results
________________________________________

🎯 WHAT IT TESTS:
* Test 2 validates that basic RAG retrieval works correctly when you query with exact or near-exact category names.
* Think of it as: Testing that searching for "dining" actually returns the "Dining" category (not something random).
________________________________________
🔍 THE 8 TEST QUERIES:

* Test 2 runs 8 exact name queries to verify retrieval:

 
4 GROUP QUERIES:
* "dining" → Expected: CG800 (Dining)
* "groceries" → Expected: CG10000 (Groceries)
* "transportation" → Expected: CG100 (Transportation)
* "healthcare" → Expected: CG300 (Healthcare & Medical)
  
4 SUBCATEGORY QUERIES:
* "coffee shops" → Expected: C806 (Cafes & Coffee Shops)
* "gas station" → Expected: C101 (Gas Station)
* "restaurants" → Expected: C803 (Restaurants)
* "pharmacy" → Expected: C302 (Pharmacy)


In [ ]:
# from tests.test_rag_1_infrastructure import test_exact_name_retrieval

# # Set YOUR threshold (or use default 0.6)
# threshold = 0.5  # ← YOU decide this value

# # Run Test 2 with YOUR threshold
# results = test_exact_name_retrieval(similarity_distance_threshold=threshold)

# # Check results
# print(f"\n'Test 2: Exact Name Retrieval'\nResult: {results['passed']}/{results['total']} queries correct")
# print(f"Accuracy: {results['accuracy']*100:.1f}%")

## RAG - TEST 3: Semantic Similarity (Broad vs Specific)

THE CRITICAL TEST that validates your core innovation! 🎯

**Part A: Broad Terms → Groups (10 tests)**
* Abstract, domain-level terms like: 
* "medicine" → Healthcare & Medical GROUP
* "food shopping" → Groceries GROUP
* "eating out" → Dining GROUP

---

**Part B: Specific Terms → Subcategories (10 tests)**
* Concrete, transaction-level terms like: 
* "dentist" → Dental SUBCATEGORY
* "coffee shop" → Cafes & Coffee Shops SUBCATEGORY
* "fuel" → Gas Station SUBCATEGORY
* Success: ≥90% accuracy
---

🎯 Why This Test Matters

**This validates the SEMANTIC UNDERSTANDING that distinguishes your RAG from simple keyword matching**:

* ❌ Without rich descriptions:
    * "medicine" → might match Pharmacy (wrong!)
      

* ✅ With rich descriptions:
    * "medicine" (broad/abstract) → Healthcare GROUP ✓
    * "dentist" (specific/concrete) → Dental SUBCATEGORY ✓

---

✅ SUCCESS CRITERIA:

* PART A: Broad terms match GROUPS (not subcategories)
* PART B: Specific terms match SUBCATEGORIES (not groups)


In [ ]:
# from tests.test_rag_2_semantic_quality import test_semantic_similarity_broad_vs_specific

# # Set YOUR threshold
# threshold = 0.4  # ← YOU control this

# # Run Test 3 with YOUR threshold
# results = test_semantic_similarity_broad_vs_specific(similarity_distance_threshold=threshold)

# # Check results
# print(f"TEST 3: Semantic Similarity (Broad vs Specific)\n📊 RESULTS:")
# print(f"Broad → Groups: {results['broad_results']['accuracy']*100:.1f}%")
# print(f"Specific → Subcats: {results['specific_results']['accuracy']*100:.1f}%")
# print(f"Overall: {results['combined']['overall_accuracy']*100:.1f}%")

## RAG - TEST 4: Distance Threshold Filtering
* Component: trn_category_tool.py → distance filtering
---
* Validates:
    * Default threshold 0.6
    * Good matches pass threshold (distance < 0.6)
    * Poor matches filtered out (distance ≥ 0.6)
    * Confidence levels assigned correctly: 
        * < 0.4 = "high"
        * 0.4-0.6 = "medium"
        * ≥ 0.6 = "low" (filtered)
---
* Think of it as: **Testing the quality filter that prevents bad matches from reaching LLM-1**

In [ ]:
# from tests.test_rag_3_tool_integration import test_distance_threshold_filtering

# # Run Test 4 (no parameters - fixed test)
# results = test_distance_threshold_filtering()

# # Check results
# print(f"\n TEST 4: Distance Threshold Filtering\nResult: {results['passed']}/{results['total']} checks passed")
# print(f"Accuracy: {results['accuracy']*100:.1f}%")

## RAG-TEST 5: Tool Wrapper Integration 🎯

* Test 5 validates that the tool wrapper produces correctly structured output for LLM-1 Router.
* Think of it as: **Testing the data format/structure that LLM-1 will receive.**

---
🔍 THE 4 CHECKS:
* CHECK 1: Returns List[CategoryMatch]
    * Verify return type is a list
    * Verify items are CategoryMatch objects


* CHECK 2: CategoryMatch has all required fields
    * user_term
    * category_id
    * category_name
    * category_type
    * distance
    * confidence
    * Optional: group_id, group_name


* CHECK 3: Results sorted by distance (best first)
    * First result has lowest distance
    * Results in ascending distance order


* CHECK 4: Confidence levels are valid
    * Must be one of: "high", "medium", "low"
    * No invalid values

In [ ]:
# from tests.test_rag_3_tool_integration import test_tool_wrapper_integration

# # Run Test 5 (no parameters - fixed test)
# results = test_tool_wrapper_integration()

# # Print formatted result
# print(f"\n TEST 5: Tool Wrapper Integration\nResult: {results['passed']}/{results['total']} checks passed")
# print(f"Accuracy: {results['accuracy']*100:.1f}%")

## RAG-TEST 6: Batch Query Processing

* **Test 6 validates that the tool can efficiently process multiple terms in a single query.**
* Think of it as: Testing the batch processing feature we just implemented!

---
🔍 THE 3 CHECKS:
* CHECK 1: Batch query completes without errors
    * Query: ["dining", "groceries", "gas station", "medical"]
    * Should complete successfully (no exceptions)

* CHECK 2: Returns results for all terms
    * Should return ≥ 4 results (at least one per term)
    * Validates aggregation works

* CHECK 3: All results are valid CategoryMatch objects
    * Each result is a proper CategoryMatch
    * No corrupt/invalid objects

---
✅ SUCCESS CRITERIA:

* Batch processing works without errors
* Results aggregated from all terms

In [ ]:
# from tests.test_rag_3_tool_integration import test_batch_query_processing

# # Run Test 6 (no parameters - fixed test)
# results = test_batch_query_processing()

# # Print formatted result
# print(f"\n TEST 6: Batch Query Processing\nResult: {results['passed']}/{results['total']} checks passed")
# print(f"Accuracy: {results['accuracy']*100:.1f}%")

## RAG-TEST 7: Complete Coverage 🎯
This is the FINAL RAG test!

📋 TEST 7 EXPLAINED:
🎯 WHAT IT TESTS:
Test 7 validates that the RAG system can find ALL categories in the knowledge base - comprehensive coverage test.
Think of it as: Testing that no category is "hidden" or unreachable by the RAG system.

🔍 THE TEST:
Queries all 108 categories (20 groups + 88 subcategories):
For each category:

Query using the category's exact name
Check if top result matches the expected category
Measure overall coverage percentage

Example:

Category: "Dining" (CG800)
Query: "Dining"
Expected: Top result = CG800


✅ SUCCESS CRITERIA:

≥80% coverage (86/108 categories found)
Groups and subcategories both covered
No systematic gaps

In [ ]:
# from tests.test_rag_4_complete_coverage import test_complete_category_coverage

# # Set YOUR threshold
# threshold = 0.6  # ← YOU control this

# # Run Test 7 with YOUR threshold
# results = test_complete_category_coverage(similarity_distance_threshold=threshold)

# # Print formatted result
# print(f"\n TEST 7: Complete Category Coverage\nResult: {results['passed']}/{results['total']} categories found")
# print(f"Coverage: {results['accuracy']*100:.1f}%")

## RAG-TEST 8: Router LLM-1 Tool Binding

* **Component:** graph_definition.py → router_llm.bind_tools() + router_node() tool loop
* **Validates:**
    * .bind_tools() was applied correctly to router_llm
    * Iterative tool calling loop works
    * Tool results fed back to LLM without error
    * RouterOutput produced (not exception/crash)

---

* **NOT Testing** (tested separately):
    * Embedding quality (test_rag_2)
    * Correct category matching (test_rag_2)
    * resolved_trn_categories population (Step 2 - prompt update)

---

* Think of it as: **Testing the PLUMBING between LLM-1 and Category RAG tool**

In [ ]:
from tests.test_rag_3_tool_integration import test_router_llm_tool_binding

# Run Test 8
results = test_router_llm_tool_binding()

# Check results
print(f"\n TEST 8: Router LLM Tool Binding")
print(f"Result: {results['passed']}/{results['total']} checks passed")
print(f"Accuracy: {results['accuracy']*100:.1f}%")

# *********************************** END TO END PIPELINE TESTS _WITH_ RAG *********************************
# LLM-1+LLM-2 for Use-Cases 04 (must RAG usage): 
---

WHAT THE TEST SHOWS:

🔹 RAG TOOL EXECUTION:
   - Was the tool called?
   - What category was found?
   - Expected vs Actual comparison

🔹 LLM-1 OUTPUT:
   - Clarity correct?
   - UC-04 detected?
   - resolved_trn_categories populated?

🔹 LLM-2 INPUT CHECK:
   - Did LLM-2 receive the category?
   - Did LLM-2 receive the dates?

🔹 LLM-2 GROUNDING CHECK:
   - Did LLM-2 use ONLY parameters from LLM-1?
   - No hallucinated filters?

🔹 LLM-2 TOOL USAGE:
   - Which tables accessed?
   - What filters applied (SQL-like)?
   - What aggregations used?

🔹 BACKOFFICE LOG:
   - Complete audit trail
   - Reasoning steps
   - Data sources


In [10]:
from tests.pipeline_rag_tests import test_rag_pipeline

# # 1. Test ALL UC-04 queries (default):
# test_rag_pipeline()
test_rag_pipeline([3, 4, 7, 8, 9, 10, 16, 17])

# # 2. Test ALL queries in silent mode:
# test_rag_pipeline(silent=True)

# 3. Test specific queries (verbose - see all LLM iterations):
# test_rag_pipeline([3, 7, 10])

# # 4. Test with shorter wait time:
# test_rag_pipeline(wait_seconds=5)



════════════════════════════════════════════════════════════════════════════════
🧪 RAG PIPELINE TEST - UC-04 QUERIES WITH CATEGORY RESOLUTION
════════════════════════════════════════════════════════════════════════════════
📋 Running 8 selected queries

📂 Loading Q&A mapping...
✅ Q&A mapping loaded

📊 Initializing dynamic expected calculator...
✅ Calculator ready (reference date: 2025-12-01)

📝 QUERIES TO TEST:
────────────────────────────────────────────────────────────────────────────────
   [ 3] "How much did I spend on dining last month compared to September?"
        → Expected: CG800
   [ 4] "How much did I spend on groceries last month?"
        → Expected: CG10000
   [ 7] "How much did I spend on healthcare last month?"
        → Expected: CG300
   [ 8] "Show me my gym payments this year"
        → Expected: C1701
   [ 9] "How much did I spend on utilities in November compared to October?"
        → Expected: CG200
   [10] "Show me my last pharmacy transaction"
        → Expec

[{'id': 3,
  'status': '✅ PASS',
  'passed': True,
  'checks': {'rag_called': True,
   'category_match': True,
   'clarity_correct': True,
   'uc04_detected': True,
   'categories_populated': True,
   'correct_category_passed': True,
   'dates_passed': True,
   'category_grounded': True,
   'start_date_grounded': True,
   'end_date_grounded': True,
   'fully_grounded': True,
   'tables_accessed': True,
   'filters_applied': True,
   'aggregations_used': True,
   'answer_validated': True}},
 {'id': 4,
  'status': '✅ PASS',
  'passed': True,
  'checks': {'rag_called': True,
   'category_match': True,
   'clarity_correct': True,
   'uc04_detected': True,
   'categories_populated': True,
   'correct_category_passed': True,
   'dates_passed': True,
   'category_grounded': True,
   'start_date_grounded': True,
   'end_date_grounded': True,
   'fully_grounded': True,
   'tables_accessed': True,
   'filters_applied': True,
   'aggregations_used': True,
   'answer_validated': True}},
 {'id': 7,

# **************************** END TO END PIPELINE TESTS _WITHOUT_ RAG *************************
# Full pipeline tests WITHOUT RAG (for UC01, UC02, UC03): LLM-1+LLM-2
***

**Silent Mode (silent=True (Default) => CLEAN OUTPUT)**

***

* Usage:
    * ✅ Normal testing - you just want to see results
    * ✅ Multiple queries - clean output is easier to read
    * ✅ Production-like testing - focus on LLM outputs

* Functions:
    * Test CLEAR queries (no category RAG needed)
        * test_clear_queries_no_rag_trn_categories([1, 2],  wait_seconds=10)
        * All queries: [1, 2, 5, 6]
      
    * Test VAGUE queries (no category RAG needed)
        * pipeline_tests import test_vague_queries_no_rag_trn_categories()
        * All queries: [12, 13]
***

**Verbose mode (silent=False => see all tool calls)**

***

* Usage:
    * 🐛 Debugging - something is failing and you need to see why
    * 🔍 Learning - you want to understand what happens internally
    * 🛠️ Development - testing new tools or graph nodes
* Function:
    * test_clear_queries_no_rag_trn_categories([12], wait_seconds=10)


In [11]:
from tests.pipeline_no_rag_tests import test_clear_queries_no_rag_trn_categories  # Test CLEAR queries (no category RAG needed)
from tests.pipeline_no_rag_tests import test_vague_queries_no_rag_trn_categories  # Test VAGUE queries (no category RAG needed)

### Silent Mode ("clean output")

In [12]:
# # Silent Mode: CLEAR non-category queries, test 2 queries:
test_clear_queries_no_rag_trn_categories([1, 2], wait_seconds=10)



🧪 TEST: CLEAR QUERIES WITHOUT RAG TRANSACTION CATEGORIES


Testing 2 queries: [1, 2]
Wait time: 10 seconds between queries
Silent mode: ON

📂 Loading Q&A mapping...
✅ Q&A mapping loaded
🔧 Building graph...
✅ Graph compiled and ready



----------------------------------------------------------------------------------------------------

📝 QUERY 1/2 (ID: 1)

----------------------------------------------------------------------------------------------------

👤 USER: "What is my current balance?"

🔹 LLM-1 ROUTER OUTPUT:
   ✓ Clarity: CLEAR
   ✓ Core UCs: ['UC-01']
   ✓ Primary UC: UC-01
   ✓ UC Operations:
      • UC-01: ['get_account_balance']
   ✓ Complexity Axes: []
   ✓ Resolved Dates: None (not temporal)
   ✓ Resolved Categories: None (no categories)
   ✓ Needed Tools: []
   ✓ Confidence: high
   ✓ Clarity Reason: Query requests a single direct value (current account balance) with no ambiguity. 'Current' is deterministic (means now/latest). This is a straightforward UC-01 direct lo

[{'id': 1, 'status': '✅ PASS', 'error': None},
 {'id': 2, 'status': '✅ PASS', 'error': None}]

In [13]:
# # Silent Mode: VAGUE non-category queries, test 2 queries:
test_vague_queries_no_rag_trn_categories([12, 13], wait_seconds=10)



🧪 TEST: VAGUE QUERIES WITHOUT RAG TRANSACTION CATEGORIES


Testing 2 queries: [12, 13]
Wait time: 10 seconds between queries
Silent mode: ON

📂 Loading Q&A mapping...
✅ Q&A mapping loaded
🔧 Building graph...
✅ Graph compiled and ready



----------------------------------------------------------------------------------------------------

📝 QUERY 1/2 (ID: 12)

----------------------------------------------------------------------------------------------------

👤 USER: "Show me recent transactions"

🔹 LLM-1 ROUTER OUTPUT (VAGUE):
   ✓ Clarity: VAGUE
   ✓ Core UCs: ['UC-03', 'UC-01']
   ✓ Primary UC: UC-03
   ❓ Clarifying Question:
      "What time period do you mean by 'recent'? For example: last 7 days, last 30 days, or this month?"
   ❓ Missing Info: ['time_window']
   ✓ Clarity Reason: Query contains subjective temporal term 'recent' without definition in conversation_summary. This is a user-preference dependent term that requires clarification.

✅ Correctly identified as VAGUE with

[{'id': 12, 'status': '✅ PASS', 'error': None},
 {'id': 13, 'status': '✅ PASS', 'error': None}]

### Verbose mode (see ALL internal logs, including Tool Calling Loop)

In [ ]:
# # Silent Mode: CLEAR non-category queries, test 2 queries:
# test_clear_queries_no_rag_trn_categories([1], silent=False) #, wait_seconds=10)

# # Silent Mode: VAGUE non-category queries, test 2 queries:
# test_vague_queries_no_rag_trn_categories([12, 13],  silent=False, wait_seconds=10)